In [7]:
import pandas as pd
import numpy as np
df = pd.read_csv("GlobalSheet.csv")
# df = pd.read_csv("GlobalSheet2.csv",header=1)

cols = []
for i in range(29):
    cols.append("feature" + str(i+1))
# print(cols)

ratings = df.drop(["Sentence Index","Ground Truth Labels"] + cols,axis=1)
groundTruth = df["Ground Truth Labels"]
users = ratings.columns
# print(ratings)

In [15]:
df.head(20)

,Sentence Index,Ground Truth Labels,wnv645,dmr693,klr543,rvm681,mnf573,cbs154,xnq518,xlp113,...,feature20,feature21,feature22,feature23,feature24,feature25,feature26,feature27,feature28,feature29
0,0,0,2,1,2,2,2,2,2,3,...,not much of anything,no movement,"""not much of anything""","""not much of anything""",not much of anything,not much of anything,not much of anything,no yuks,not much,not much of anything
1,1,0,3,4,1,1,1,1,1,1,...,retch it up like rancid,retch,"""drivel"", ""sickly"", ""wretch""","""retch it up like rancid crème brûlée""",retch,a gob of drivel so sickly sweet,will retch it up,so sickly sweet,sickly sweet,"drivel, retch it upp"
2,2,0,3,2,1,1,1,3,1,1,...,unapologetic mess,mess,"""only saving grace""","""unapologetic mess""",an unapologetic mess,unapologetic mess,an unapologetic mess,unapologetic mess,unapologetic mess,unapologetic mess
3,3,0,2,3,2,2,2,2,1,2,...,never really feel involved,never really feel involved,"""never really feel involved""","""never really feel involved"", ""abstract ideas""",NaN,never really feel involved,never really feel involved,abstract ideas,never feel involved,never feel involved
4,4,1,5,5,4,5,5,5,5,5,...,best films,best,"""best films""",""" best films""",best films,best,one of polanski 's best films,best films,best,one of best films
5,5,1,5,4,4,4,4,5,4,4,...,refreshingly different,refreshingly,"""refreshingly different""",""" refreshingly different""",refreshingly different,refreshingly,a refreshingly different,refreshingly,refreshingly different,refreshingly
6,6,0,2,3,2,2,3,2,2,4,...,NaN,almost worthwhile,"""almost makes ''never again'' worthwhile""","""almost makes `` never again '' worthwhile, but """,NaN,"almost, but","almost makes `` never again '' worthwhile , but",worthwhile,never again,almost makes worthwhile
7,7,1,3,4,4,4,4,4,4,4,...,NaN,fine acting,"""its fine acting""","""fine acting""",fine acting,fine acting,fine acting,fine acting,fine,fine acting
8,8,0,2,3,2,2,2,2,2,2,...,will leave frowns on more than a few faces,frowns,"""will leave frowns""","""leave frowns on more than a few faces""",NaN,frowns,will leave frowns on more than a few faces,frowns,frowns,leave frown
9,9,0,1,1,1,1,1,1,1,1,...,if this holiday movie is supposed to be a gift...,crap,"""took out all the good stuff , and left behind...","""took out all the good stuff , and left behind...",NaN,good stuff,"took out all the good stuff , and left behind ...",crap,crap,"took out all the good stuff, the crap"


In [8]:
trustRank = pd.DataFrame(columns=users)
trustRank.loc[0] = [0] * ratings.shape[1]

numOfItems = ratings.shape[0]
numOfUsers = ratings.shape[1]

# itemRateOfAll = pd.DataFrame(columns=ratings.columns)
# itemRateOfAll.loc[0] = [0]*ratings.shape[1]
itemRateOfAll = [0] * numOfItems

# Is the item i.th item got to a decidable state
converged = [False] * numOfItems

# Convergens bounds
lowerBound = 1
upperBound = 4

# Trust score minimum
untrusted = 0.8

# Number of iterations
iterations = 50

# Do all converged?
def conv():
    for c in converged.values:
        if not c:
            return False
    return True

# Trust rate recalculator
def calcRate(rates):
    new = 0
    trusted = 0
    for i, rate in enumerate(rates):
        # print(trustRank.values[0][i], rate)
        if rate < 3 and trustRank.values[0][i] > untrusted:
            new += trustRank.values[0][i] * 0
            trusted += 1
        elif trustRank.values[0][i] > untrusted:
            new += trustRank.values[0][i] * 5
            trusted += 1
    return new / trusted

# First just calculate the means of the rankings
for i, row in ratings.iterrows():
        # Calc new mean
        meanRate = round(np.mean(row),2)
        itemRateOfAll[i] = meanRate


# print(itemRateOfAll)
# Iterate to converge with trust
for z in range(iterations):
    # Recap the users' trustScore
    # Go throw Users
    for userName, itemRanksOfUser in ratings.iteritems():
        # print('User',userName)
        # print('itemRanks',itemRanksOfUser.values)
        # Trust sum (0/1 | )
        trust = 0
        # Go throw each Item with each user
        for itemID, rate in enumerate(itemRanksOfUser.values):
            # print(itemID, 'th item rated with:', rate, 'by', user)
            
            # WE MIGHT WANNA USE THE GROUND TRUTH HERE (here worst result than the current)
            # if groundTruth[itemID] == 0 and rate < 3:
            if itemRateOfAll[itemID] < 3 and rate < 3:
                trust += 1
            # if groundTruth[itemID] == 1 and rate >= 3:
            if itemRateOfAll[itemID] >= 3 and rate >= 3:
                trust += 1
            # trust += abs(rate - itemRateOfAll[k])
        trustRank[userName][0] = trust / numOfItems
    # print(trustRank)
    # Recalculate with the new trust scores
    for i, row in ratings.iterrows():
        # Calc new mean
        meanRate = round(calcRate(row),2)
        # If good enough end the iterations on that
        if not converged[i] and meanRate < lowerBound:
            itemRateOfAll[i] = 0
            converged[i] = True
            # print("low")
        elif not converged[i] and meanRate > upperBound:
            itemRateOfAll[i] = 5
            converged[i] = True
            # print("big")
        elif not converged[i]:
            itemRateOfAll[i] = meanRate
    if False not in converged:
        print("All item converged",z)
        break

In [28]:
decided = 0
for d in converged:
    if d:
        decided += 1
print('Clear decsions:', decided ,'out of', numOfItems)
# print(str(numOfItems - decided) ,'number of reviews were decided by there state after the iterations.')


# If there haven't converge, we decide it at that point
for i, rate in enumerate(itemRateOfAll):
    if rate != 0 or rate != 5:
        if rate < 3:
            itemRateOfAll[i] = 0
        if rate >= 3:
            itemRateOfAll[i] = 5

# The crowd's decision
finalRatings = [int(x / 5) for x in itemRateOfAll]
# print(finalRatings)
# print(converged)
# print(trustRank)

# The error of the crowd was 5%
result = round((numOfItems - sum(abs(groundTruth - finalRatings))) / numOfItems,2)
print('Matc with ground truth:', numOfItems - sum(abs(groundTruth - finalRatings)))
print('Precision: ' + str(result*100) + '%.')

Clear decsions: 82 out of 101
Matc with ground truth: 96
Precision: 95.0%.


101
101
